In [ ]:
!pip install tensorflow
!pip install opencv-python
!pip install opencv-contrib-python
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the CapsNet model
def CapsuleNetwork(input_shape, num_classes):
    input_layer = keras.layers.Input(shape=input_shape)

    # Convolutional layer
    conv1 = layers.Conv2D(256, (9, 9), activation='relu')(input_layer)

    # Primary Capsules layer
    primary_capsules = layers.Conv2D(256, (9, 9), activation='relu')(conv1)

    # Digit Capsules layer
    digit_capsules = layers.Conv2D(num_classes * 16, (9, 9))(primary_capsules)
    out_capsules = layers.Reshape((-1, num_classes))(digit_capsules)

    # Capsule output layer
    capsule_output = layers.Lambda(lambda x: keras.backend.sqrt(keras.backend.sum(keras.backend.square(x), axis=-2)))(out_capsules)

    # Create the model
    model = keras.models.Model(inputs=input_layer, outputs=capsule_output)

    return model

# Instantiate the Capsule Network model
input_shape = (225, 225, 3)  # Adjust input shape according to your images
num_classes = 2  # Two classes: Fire and No Fire
model = CapsuleNetwork(input_shape, num_classes)


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/forest_fire/Training and Validation/', target_size=input_shape[:2], batch_size=32, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory('/content/drive/MyDrive/forest_fire/Testing/', target_size=input_shape[:2], batch_size=32, class_mode='categorical')

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=3,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)


Found 1832 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Epoch 1/3
57/57 [==============================] - 625s 8s/step - loss: 0.3158 - accuracy: 0.8694 - val_loss: 0.4379 - val_accuracy: 0.7656
Epoch 2/3
57/57 [==============================] - 233s 4s/step - loss: 0.2037 - accuracy: 0.9356 - val_loss: 0.3234 - val_accuracy: 0.8281
Epoch 3/3
57/57 [==============================] - 231s 4s/step - loss: 0.1849 - accuracy: 0.9344 - val_loss: 0.3415 - val_accuracy: 0.8594


In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // 32)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')


2/2 [==============================] - 1s 370ms/step - loss: 0.3593 - accuracy: 0.8438
Validation Loss: 0.3593, Validation Accuracy: 0.8438


In [ ]:

# Define the path where you want to save the model on Google Drive
model_save_path = '/content/drive/My Drive/path_to_save_model.keras'

# Save the model
model.save(model_save_path)

# Check if the model is saved
import os
os.path.exists(model_save_path)


True

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Define a function to predict fire presence using a model file path
def predict_fire_from_image(image_path, model_path):
    # Load the pre-trained model
    model = load_model(model_path)

    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(225, 225))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the pixel values to the range [0, 1]

    # Make the prediction
    prediction = model.predict(img)

    # Assuming your model has two classes: Fire and No Fire
    class_names = [ 'Fire','No Fire']

    # Get the class label with the highest probability
    predicted_class = np.argmax(prediction, axis=1)
    confidence = prediction[0][predicted_class[0]]

    # Get the corresponding class name
    predicted_class_name = class_names[predicted_class[0]]

    return predicted_class_name, confidence

# Example usage:
image_path = '/content/drive/MyDrive/forest_fire/Training and Validation/fire/abc008.jpg'
model_path = '/content/drive/MyDrive/path_to_save_model.h5'  # Replace with the path to your trained model

predicted_class, confidence = predict_fire_from_image(image_path, model_path)
print(f'The image is predicted as "{predicted_class}" with confidence {confidence:.2f}')


1/1 [==============================] - 0s 65ms/step
The image is predicted as "Fire" with confidence 1484958.12
